In [1]:
!pip install -q pyspellchecker

     |████████████████████████████████| 1.9MB 2.8MB/s 


In [2]:
import numpy as np
import re
import nltk
from nltk.corpus import stopwords

#from spellchecker import SpellChecker

import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding, LSTM, Bidirectional
from keras.utils import np_utils

Using TensorFlow backend.


In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [70]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Loading the data

Training data

In [0]:
f = open('/content/drive/My Drive/Projects/Premoji/Data/Train/tweets.txt.text', 'r')
#tweets_train = f.readlines()[:50000] # using 10% of the actual data for fast training#
tweets_train = f.readlines()
f.close()

In [73]:
f = open('/content/drive/My Drive/Projects/Premoji/Data/Train/tweets.txt.labels', 'r')
#labels_train = f.readlines()[:50000] # using 10% of the actual data for fast training
labels_train = f.readlines()
f.close

<function TextIOWrapper.close>

In [0]:
y_train = np_utils.to_categorical([int(l.strip()) for l in labels_train], 20)

Test data

In [0]:
f = open('/content/drive/My Drive/Projects/Premoji/Data/Test/us_test.text', 'r')
tweets_test = f.readlines()
f.close()

In [76]:
f = open('/content/drive/My Drive/Projects/Premoji/Data/Test/us_test.labels', 'r')
labels_test = f.readlines()
f.close

<function TextIOWrapper.close>

In [0]:
y_test = np_utils.to_categorical([int(l.strip()) for l in labels_test], 20)

# Preprocessing the text

## Cleaning the text

In [0]:
def clean_tweets(tweets):
  # Stripping away location for the end of tweets
  p = re.compile(r'\s*@ .*$')
  tweets_1 = [p.sub('', tweet) for tweet in tweets]
  
  # Removing @user mentions
  tweets_2 = [tweet.replace('@user', '') for tweet in tweets_1]
  
  # Removing stopwords
  stop_words = set(stopwords.words('english'))

  tweets_3 = []

  for tweet in tweets_2:
    res = ''
    for w in tweet.split():
      if w.lower() not in stop_words:
        res = res+w+' '
    tweets_3.append(res)
    
  # Removing unicode characters
  tweets_4 = [(tweet.encode('ascii', 'ignore')).decode("utf-8") for tweet in tweets_3]
  
  return tweets_4
  
  # Spell checking
#   spell = SpellChecker()
  
#   tweets_5 = []

#   for tweet in tweets_4:
#     res = ''
#     for w in tweet.split():
#       correction = spell.correction(w.lower())
#       res = res+correction+' '
#       print(w+";"+correction)
#     tweets_5.append(res)
    
#   return tweets_5

Cleaning training data

In [0]:
tweets_train = clean_tweets(tweets_train)

Cleaning test data

In [0]:
tweets_test = clean_tweets(tweets_test)

## Tokenizing the text

In [0]:
vocab_size = 50000
tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', num_words=vocab_size, oov_token="UNK")
tokenizer.fit_on_texts(texts=tweets_train)

In [0]:
#tokenizer.get_config()

Encoding and adding a padding to the training data

In [0]:
tokenized_train = tokenizer.texts_to_sequences(tweets_train)
x_train = pad_sequences(tokenized_train, padding='post')

Retrieving unified vector length after padding

In [0]:
maxlen = len(x_train[0])

Encoding and adding a padding to the test
data

In [0]:
tokenized_test = tokenizer.texts_to_sequences(tweets_test)
x_test = pad_sequences(tokenized_test, padding='post', maxlen=maxlen)

# Word2vec

Importing word2vec

In [0]:
from gensim.models import Word2Vec

Training the embedding

In [0]:
sentences = [sentence.split() for sentence in tokenizer.sequences_to_texts(tokenized_train)]

In [0]:
w2v = Word2Vec(sentences=sentences, size=200, window=5, workers=4, min_count=1)

In [0]:
#embedding = model.wv.get_keras_embedding()

In [0]:
from numpy import zeros

embedding_matrix = zeros((vocab_size, 200))
for word, i in tokenizer.word_index.items():
  try:
    embedding_vector = w2v.wv.get_vector(word)
  except:
    continue
  embedding_matrix[i] = embedding_vector

# Training the model

Building the model

In [0]:
model = Sequential()

In [0]:
#model.add(Embedding(input_dim=vocab_size, outpub_dim=200))
model.add(Embedding(input_dim=vocab_size, output_dim=200, weights=[embedding_matrix], trainable=False))
model.add(LSTM(100))
#model.add(Dropout(rate=0.5))
model.add(Dense(20, activation='softmax'))

In [0]:
model.compile(loss='categorical_crossentropy',
             optimizer='rmsprop',
             metrics=['accuracy'])

Training the model

In [93]:
model.fit(x_train, y_train,
         validation_split=0.3, epochs=10, batch_size=64)

Train on 340005 samples, validate on 145717 samples
Epoch 1/10
340005/340005 [==============================] - 262s 771us/step - loss: 2.3892 - acc: 0.3056 - val_loss: 2.3217 - val_acc: 0.3206
Epoch 2/10
340005/340005 [==============================] - 260s 766us/step - loss: 2.3003 - acc: 0.3248 - val_loss: 2.2873 - val_acc: 0.3294
Epoch 3/10
340005/340005 [==============================] - 259s 763us/step - loss: 2.2649 - acc: 0.3323 - val_loss: 2.2551 - val_acc: 0.3355
Epoch 4/10
340005/340005 [==============================] - 256s 754us/step - loss: 2.2411 - acc: 0.3367 - val_loss: 2.2437 - val_acc: 0.3391
Epoch 5/10
340005/340005 [==============================] - 252s 743us/step - loss: 2.2224 - acc: 0.3413 - val_loss: 2.2322 - val_acc: 0.3407
Epoch 6/10
340005/340005 [==============================] - 255s 751us/step - loss: 2.2057 - acc: 0.3447 - val_loss: 2.2300 - val_acc: 0.3393
Epoch 7/10
340005/340005 [==============================] - 258s 760us/step - loss: 2.1907 - acc

# Evaluation

In [94]:
loss, accuracy = model.evaluate(x_test, y_test)

50000/50000 [==============================] - 18s 355us/step


Results :

In [95]:
print('Loss :', loss)
print('Accuracy :', accuracy)

Loss : 2.3248942930603027
Accuracy : 0.32928


# Prediction

In [0]:
tweet = "RuPaul's Drag Race bingo fun. Drag Queens be SEXY! #rupaulsdragrace @user abwyman #la…"
label = 9

tokens = tokenizer.texts_to_sequences([tweet])
x = pad_sequences(tokens, padding='post', maxlen=maxlen)

y = np_utils.to_categorical([label], 20)

print(x[0])
print(y)

[    1  2927  1271  6326    29  2927  1925   507   817  6803 24953     1
     1     0     0     0     0     0     0     0     0     0     0     0]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [0]:
prediction = model.predict(x)

print(range(1,21)[np.argmax(prediction[0])])

2


# Saving the model

In [0]:
model.save('/content/drive/My Drive/Projects/Premoji/lstm.h5')